We all know the Importance of good features for machine learning models. In Machine learning task we have features which we need to process to make them good and this is done by data preprocessing tasks.

Data Preprocessing : Data preprocessing is a process of preparing the raw data and making it suitable for a machine learning model. It is the first and crucial step while creating a machine learning model. When creating a machine learning project, it is not always a case that we come across the clean and formatted data. And while doing any operation with data, it is mandatory to clean it and put in a formatted way. So for this, we use data preprocessing task.

A real-world data generally contains noises, missing values, and maybe in an unusable format which cannot be directly used for machine learning models. Data preprocessing is required tasks for cleaning the data and making it suitable for a machine learning model which also increases the accuracy and efficiency of a machine learning model.

Data Preprocessing involves below steps:

    Getting the dataset
    Importing libraries
    Importing datasets
    Finding Missing Data
    Encoding Categorical Data

Many of us know traditional approaches for above listed steps but in this notebook I will discuss some different approaches which could be game changer for your next project.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/loan-default-dataset/Loan_Default.csv")
df.head()

In [ ]:
df.info()

Missing data are not rare in real data sets. In fact, the chance that at least one data point is missing increases as the data set size increases. Missing data can occur any number of ways, some of which include the following.

    Merging of source data sets
    Random events
    Failures of measurement


## 2.1 Visualizing Missing Data
Visualizations as well as numeric summaries are the first step in understanding the challenge of missing information in a data set. For small to moderate data (100s of samples and 100s of attributes), several techniques are available that allow the visualization of all of the samples and Attributes simultaneously.

In this notebook I'll Cover Following visualizations for missing values:-

    Matrix
    Correlation Heatmap
    Dendrogram
    Simple numerical summaries

Question may arise that why we need Visualizations? Because it is wise to explore relationships within the attributes that might be related to missingness.

## 2.1.1 Matrix

It is the nullity matrix that allows us to see the distribution of data across all columns in the whole dataset. It also shows a sparkline (or, in some cases, a striped line) that emphasizes rows in a dataset with the highest and lowest nullity.

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(df)
plt.figure(figsize = (15,9))
plt.show()  

From the above plot we can interpret our dataset has lots of missing values in it

##  2.1.2 Correlation Heatmap 
Correlation heatmap measures nullity correlation between columns of the dataset. It shows how strongly the presence or absence of one feature affects the other.

Nullity correlation ranges from(-1 to 1):

    -1 means if one column(attribute) is present, the other is almost certainly absent.
    0 means there is no dependence between the columns(attributes).
    1 means if one column(attributes) is present, the other is also certainly present.

Unlike in a familiar correlation heatmap, if you see here, many columns are missing. Those columns which are always full or always empty have no meaningful correlation and are removed from the visualization.

The heatmap is helpful for identifying data completeness correlations between attribute pairs, but it has the limited explanatory ability for broader relationships and no special support for really big datasets.

In [ ]:
msno.heatmap(df, labels = True)

From above visualization we can easily interpret missingness of attribute rate_of_interest and upfront_charges is dependent on each other(correlation value = 1) means if one will be present another will be present.

##  2.1.3 Dendrogram 
The dendrogram shows the hierarchical nullity relationship between columns. The dendrogram uses a hierarchical clustering algorithm against one another by their nullity correlation.

More about Hierarchical Clustering Algorithm

In [ ]:
# Columns having missing values
missing_columns = [col for col in df.columns if df[col].isnull().sum() > 0]
missing_columns

In [ ]:
msno.dendrogram(df[missing_columns])

We interpret the dendrogram based on a top-down approach, i.e., to focus on the height at which any two columns are joined together with matters of nullity. More will be the height less will be the relation and vice versa is also True.

For example if we see pair of attributes LTV and property value has height 0 implies they are highly correlated in case of nullity. Similarly attribute LTV and rate_of_interest have maximum height implies they are less correlated with each other.


## 2.1.4 Simple Numerical Summaries 

Moving Forward lets try to analyse numerical summary of missing attributes. Simple numerical summaries are effective at identifying problematic predictors and samples when the data become too large to visually inspect.

In [ ]:
def get_numerical_summary(df):
    total = df.shape[0]
    missing_columns = [col for col in df.columns if df[col].isnull().sum() > 0]
    missing_percent = {}
    for col in missing_columns:
        null_count = df[col].isnull().sum()
        per = (null_count/total) * 100
        missing_percent[col] = per
        print("{} : {} ({}%)".format(col, null_count, round(per, 3)))
    return missing_percent

In [ ]:
missing_percent = get_numerical_summary(df)

Now I guess visualization part is done lets move forward to methods which we can use to handle missing values.


##  2.2 Methods to Handle Missing Data

As we Know if our data has missing values than our model will not train except few models which can tolerate them like some tree based models but the point is we want to handle this and how can we handle them. So, in this notebook to handle missing data I will discuss following techniques :-

    Deletion of Data
    Encoding Missingness
    Imputation Methods


### 2.2.1 Deletion of Data 

The simplest approach for dealing with missing values is to remove entire attribute(s) and/or sample(s) that contain missing values. However, one must carefully consider a number of aspects of the data prior to taking this approach. For example, missing values could be eliminated by removing all predictors that contain at least one missing value. Similarly, missing values could be eliminated by removing all samples with any missing values.

Note: When it is difficult to obtain samples or when the data contain a small number of samples (i.e., rows), then it is not desirable to remove samples from the data.

Consider this small intuition shown below

Let M = Number of Samples(rows).\ and Let N = Number of Attributes(columns).

Case 1: Deletion of Attributes

If N has range of [1-10]\ Then don't delete the attribute that contain missing values but if that attribute has missing values around 80-90% then deletion of that attribute will be good option instead of just predicting values of those 80-90% data based on that 10-20% data.

Case 2: Deletion of Samples

If M is a large number according to your task\ Then deletion of sample can be a Good step but if that sample has few missing values with respect to attribute, then you should consider methods to fill those missing values.

Lets move on to the implementation part, I will just show how to delete data for both cases but you can interpret more according to your tasks.

##### Deletion of an Attribute

According to Simple numerical Summaries the attribute Upfront_charges has largest missing values percentage of (26.664%) which is not ideal percentage to remove a feature but just for sake of implementation I will remove that feature.

df_temp = df.copy()

In [ ]:
# Threshold to remove attribute having missing values greater than threshold
ATTRIBUTE_THRESHOLD = 25 #25% in this case 

for col, per in missing_percent.items():
    if per > ATTRIBUTE_THRESHOLD:
        df_temp.drop(col, axis = 1, inplace = True)

By generating numerical summary of df_temp we can see now attribute Upfont_chargers being removed from the dataset as it has missing values percentage greater than threshold we defined


In [ ]:
_ = get_numerical_summary(df_temp)

In [ ]:
del df_temp

### Deletion of Samples
We will try to delete those samples having missing values in more than 5 attributes

In [ ]:
df_temp = df.copy()

In [ ]:
# Getting Missing count of each sample            

for idx in range(df_temp.shape[0]):
    df_temp.loc[idx, 'missing_count'] = df_temp.iloc[idx, :].isnull().sum()  

In [ ]:
# Threshold to remove samples having missing values greater than threshold
SAMPLE_THRESHOLD = 5

print("Samples Before Removal : {}".format(df_temp.shape[0]))

df_temp.drop(df_temp[df_temp['missing_count'] > SAMPLE_THRESHOLD].index, axis = 0, inplace = True)

print("Samples After Removal : {}".format(df_temp.shape[0]))


In [ ]:
del df_temp